In [41]:
import tensorflow as tf

In [42]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [43]:
input_width = 28
input_length = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
conv1_stride = 1
conv2_stride = 1
max_pool1_k = 2
max_pool2_k = 2
conv1_k = 5
conv2_k = 5
n_hidden = 1024
n_out = 10

input_hidden = (input_width//(max_pool1_k*max_pool2_k))*(input_length//(max_pool1_k*max_pool2_k))*n_conv2

In [44]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh" : tf.Variable(tf.random_normal([input_hidden,n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden,n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out])),
}

In [45]:
def conv(x,weight,bias,stride = 1):
    
    out = tf.nn.conv2d(x, weight,padding="SAME",strides= [1,stride,stride,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out 

def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [46]:
def cnn(x,weights,biases,keep_prob):
    x = tf.reshape(x,shape = [-1,input_length,input_width,input_channels])
    
    conv1 = conv(x,weights["wc1"],biases["bc1"],conv1_stride)
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights["wc2"],biases["bc2"],conv2_stride)
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape = [-1,input_hidden])
    hidden_out_pre_act = tf.add(tf.matmul(hidden_input,weights["wh"]),biases["bh"])
    hidden_out_pre_drop = tf.nn.relu(hidden_out_pre_act)
    hidden_out = tf.nn.dropout(hidden_out_pre_drop,keep_prob)
    output = tf.add(tf.matmul(hidden_out,weights["wo"]),biases["bo"])
    
    return output

In [48]:
x = tf.placeholder("float",[None,input_pixels])
y = tf.placeholder(tf.int32,[None,n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x,weights,biases,keep_prob)

In [49]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

In [50]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [51]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

853406.2098654509
32940.18437627703
18613.250762220356
12294.787240054931
9869.14270253235
7167.140495563118
7233.3169985297045
7560.514638606459
7258.493033144746
6331.79700681398
5285.155256273807
4583.0361046430435
4358.784153383768
4125.066263819113
3913.453856447789
3469.103511014506
3259.1944598938353
3027.718316535091
2461.147716818089
2872.0122359619854


In [ ]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_predictions  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels,keep_prob:1.0})
correct_predictions.sum()